In [1]:
import os
import geopandas as gpd
import pandas as pd
import gc

In [2]:
%load_ext autoreload
import icebath as icebath
from icebath.core import bathy_workflow
%autoreload 2

In [ ]:
# import dask
# from dask.distributed import Client, LocalCluster
# client = Client(processes=False) #, n_workers=2, threads_per_worker=3, memory_limit='6GB')
# client

In [3]:
# set up variables to run
# inpardir = '/Users/jessica/projects/bathymetry_from_bergs/DEMs/2m/'
# fjord = 'JI'
# metastr='_mdf'

inpardir = '/Users/jessica/projects/bathymetry_from_bergs/DEMs/KaneWV03/'
fjord="KB"
metastr="_meta"

outdir = "/Users/jessica/projects/bathymetry_from_bergs/results/"

In [4]:
dirlist = [d for d in os.listdir(inpardir) if not d.startswith('.')]

In [5]:
print(dirlist)

['WV03_20150918_1040010011A60B00_1040010011793300_2m_lsf_v030104', 'WV03_20160807_10400100206E0000_1040010021260C00_2m_lsf_v030104', 'WV03_20170530_104001002DD07F00_104001002CB70300_2m_lsf_v030208', 'WV03_20150410_104001000A9CFA00_104001000A458C00_2m_lsf_v030104', 'WV03_20160322_104001001A4FD100_104001001A9FA500_2m_lsf_v030208', 'WV03_20160715_104001001F8FEA00_104001001FA40500_2m_lsf_v030208', 'WV03_20170610_104001002EA94600_104001002E5EF000_2m_lsf_v030208', 'WV03_20180322_1040010039AC6300_1040010039ABCE00_2m_lsf_v030403', 'WV03_20161015_1040010024520700_1040010024C7DC00_2m_lsf_v030104', 'WV03_20150714_104001000E2DDC00_104001000E236A00_2m_lsf_v030104', 'WV03_20180511_104001003DA85200_104001003DAB9B00_2m_lsf_v030403', 'WV03_20160809_104001002127B300_10400100209F7000_2m_lsf_v030104', 'WV03_20170521_104001002DB6E600_104001002C578300_2m_lsf_v030208', 'WV03_20150403_104001000983D000_1040010009949B00_2m_lsf_v030000', 'WV03_20160611_104001001D5C8500_104001001DA99700_2m_lsf_v030104', 'WV03_201

In [6]:
def recursive_dir_size(path):
    size = 0

    for x in os.listdir(path):
        if not os.path.isdir(os.path.join(path,x)):
            size += os.stat(os.path.join(path,x)).st_size
        else:
            size += recursive_dir_size(os.path.join(path,x))

    # convert to GB
    size = size / (1024.0**3)
    
    return size

In [7]:
# run the bathymetry-from-iceberg-freeboard workflow on each dem (skipping larger dirs for now)
for indir in dirlist:
#     print(indir)
    try:
#         print(recursive_dir_size(inpardir+indir))
        if recursive_dir_size(inpardir+indir) < 0.75:
            bathy_workflow.run_workflow(inpardir+indir+'/', fjord, outdir, indir[0:14] + "icebergs.gpkg", metastr=metastr, bitmask=True)
        else:
            continue
        gc.collect()
    except NotADirectoryError:
        if recursive_dir_size(inpardir+indir) < 0.75:
            bathy_workflow.run_workflow(inpardir+'/', fjord, outdir, "JIicebergs.gpkg", metastr=metastr)
            break
        else:
            continue        

/Users/jessica/projects/bathymetry_from_bergs/github/icebath/icebath/core/build_xrds.py:22: UserWarning: This function currently assumes a constant grid and EPSG for all input files
  warnings.warn("This function currently assumes a constant grid and EPSG for all input files")


WV03_20150918_1040010011A60B00_1040010011793300_2m_lsf_seg1_dem.tif
Please note the transform is computed assuming a coordinate reference system    where x(min) is west and y(min) is south
Note that the new file is reprojected to have the same CRS as the dataset to which it is being added.        However, if the two CRSs are compatible, the spatial properties of the new file may be added to or overwrite the ones of the existing dataset
Note that tide model and epsg are hard coded in!
They can also be provided as keywords if the wrapper function is updated to handle them
going to enter the rasterize function
The current fjord does not have a minimum berg size entry - using a default value!
NOT USING DASK
Got labeled raster of potential icebergs for an image
done rasterizing and getting possible icebergs
27
(-414474.4528408556, -1095390.7234042552, -413802.389522284, -1094624.6132834964)
<xarray.Dataset>
Dimensions:       (x: 10614, y: 6957)
Coordinates:
  * y             (y) float64 -1.

median iceberg freeboard less than 15 m
(-421211.0875341562, -1101313.5747556067, -421105.0775464054, -1101163.5531914893)
<xarray.Dataset>
Dimensions:       (x: 10614, y: 6957)
Coordinates:
  * y             (y) float64 -1.09e+06 -1.09e+06 ... -1.104e+06 -1.104e+06
  * x             (x) float64 -4.309e+05 -4.309e+05 ... -4.097e+05 -4.097e+05
    dtime         datetime64[ns] 2015-09-18T19:24:15
    land_mask     (y, x) bool True True True True ... False False False False
    mapping       int64 0
Data variables:
    elevation     (y, x) float32 dask.array<chunksize=(1024, 1024), meta=np.ndarray>
    geoid         (y, x) int16 dask.array<chunksize=(1024, 1024), meta=np.ndarray>
    tidal_corr    float64 1.276
    min_tidal_ht  float64 -1.405
    max_tidal_ht  float64 1.276
Attributes:
    transform:     | 2.00, 0.00,-430896.00|\n| 0.00,-2.00,-1090358.00|\n| 0.0...
    crs:           +proj=stere +lat_0=90 +lat_ts=70 +lon_0=-45 +x_0=0 +y_0=0 ...
    res:           (2.0, 2.0)
    is_tiled:

median iceberg freeboard less than 15 m
(-421391.10449448787, -1102905.8036227718, -421163.0830114011, -1102739.7797584818)
<xarray.Dataset>
Dimensions:       (x: 10614, y: 6957)
Coordinates:
  * y             (y) float64 -1.09e+06 -1.09e+06 ... -1.104e+06 -1.104e+06
  * x             (x) float64 -4.309e+05 -4.309e+05 ... -4.097e+05 -4.097e+05
    dtime         datetime64[ns] 2015-09-18T19:24:15
    land_mask     (y, x) bool True True True True ... False False False False
    mapping       int64 0
Data variables:
    elevation     (y, x) float32 dask.array<chunksize=(1024, 1024), meta=np.ndarray>
    geoid         (y, x) int16 dask.array<chunksize=(1024, 1024), meta=np.ndarray>
    tidal_corr    float64 1.276
    min_tidal_ht  float64 -1.405
    max_tidal_ht  float64 1.276
Attributes:
    transform:     | 2.00, 0.00,-430896.00|\n| 0.00,-2.00,-1090358.00|\n| 0.0...
    crs:           +proj=stere +lat_0=90 +lat_ts=70 +lon_0=-45 +x_0=0 +y_0=0 ...
    res:           (2.0, 2.0)
    is_tiled

/Users/jessica/projects/bathymetry_from_bergs/github/icebath/icebath/core/build_xrds.py:22: UserWarning: This function currently assumes a constant grid and EPSG for all input files
  warnings.warn("This function currently assumes a constant grid and EPSG for all input files")


WV03_20150410_104001000A9CFA00_104001000A458C00_2m_lsf_seg2_dem.tif
Please note the transform is computed assuming a coordinate reference system    where x(min) is west and y(min) is south
Note that the new file is reprojected to have the same CRS as the dataset to which it is being added.        However, if the two CRSs are compatible, the spatial properties of the new file may be added to or overwrite the ones of the existing dataset
Note that tide model and epsg are hard coded in!
They can also be provided as keywords if the wrapper function is updated to handle them
going to enter the rasterize function
The current fjord does not have a minimum berg size entry - using a default value!
NOT USING DASK
Got labeled raster of potential icebergs for an image
done rasterizing and getting possible icebergs
15
(-416519.5001672552, -1029280.4061975303, -416319.49273369263, -1029122.402839472)
<xarray.Dataset>
Dimensions:       (x: 21341, y: 16468)
Coordinates:
  * x             (x) float64 -

NoDataInBounds: Unable to determine bounds from coordinates. Data variable: elevation

In [ ]:
reslist = [f for f in os.listdir(outdir) if f.endswith('.gpkg')]

In [ ]:
print(reslist)

In [ ]:
# combine the geopackages into one geodataframe
gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(outdir+res) for res in reslist],
                                 ignore_index=True), crs=gpd.read_file(outdir+reslist[0]).crs)

In [ ]:
gdf

In [ ]:
outfn = fjord + "_combined_results.gpkg"
gdf.to_file(outdir+outfn, driver="GPKG")

In [ ]:
from icebath.utils import plot as ibplot

In [ ]:
ibplot.meas_vs_infer_fig(gdf, save=False)